In [1]:
import lightfm
import scipy as sp
import pymongo
import pandas as pd
import numpy as np

c:\Users\Cody\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [6]:
# Connect to mongodb on 192.168.1.87
client = pymongo.MongoClient(
    host='192.168.1.87',
    port=27017)
# Get the yelp database
db = client.yelp
review = db.review

In [11]:
# Get a random sample of 1000 reviews
pipeline = [
    {
        '$project': {
            '_id': 0, 
            'user_id': '$user_id', 
            'business_id': '$business_id', 
            'stars': '$stars'
        }
    }, 
    # {
    #     '$sample': {
    #         'size': 10000
    #     }
    # }
]
df = pd.DataFrame(list(review.aggregate(pipeline)))
# Subset to only the user_id, business_id, and stars
df = df[['user_id', 'business_id', 'stars']]
# Write to parquet to save space
df.to_parquet('yelp.parquet',compression='gzip')

In [25]:
df.to_parquet('yelp.parquet',)

In [23]:
df2 = pd.read_csv('yelp.csv')


In [24]:
df2

,Unnamed: 0,user_id,business_id,stars,train,train_int
0,0,0,0,3.0,True,1
1,1,1,1,5.0,True,1
2,2,2,2,5.0,False,0
3,3,3,3,3.0,False,0
4,4,4,4,1.0,True,1
...,...,...,...,...,...,...
7012275,7012275,6302,126169,NaN,True,1
7012276,7012276,42710,4420,NaN,False,0
7012277,7012277,15461,78815,NaN,True,1
7012278,7012278,103413,106820,NaN,False,0


In [12]:
# Read in yelp.parquet
df = pd.read_parquet('yelp.parquet')
# Train test split
from sklearn.model_selection import train_test_split


# Replace user_id and business_id with unique integers
user_id = df.user_id.unique()
business_id = df.business_id.unique()
user_id_map = {user_id[i]: i for i in range(len(user_id))}
business_id_map = {business_id[i]: i for i in range(len(business_id))}
# Replace
df.user_id = df.user_id.map(user_id_map)
df.business_id = df.business_id.map(business_id_map)
# Add column to indicate train test split
df['train'] = np.random.uniform(0, 1, len(df)) <= .75
df['train_int'] = df.train.astype(int)
# Create sparse matrix of user_id, business_id, stars, and train_int


# sparse_train = sp.sparse.coo_matrix((df[df.train].stars, (df[df.train].user_id, df[df.train].business_id)))
# sparse_test = sp.sparse.coo_matrix((df[~df.train].stars, (df[~df.train].user_id, df[~df.train].business_id)))

In [19]:
sparse = sp.sparse.coo_matrix((df.stars, (df.user_id, df.business_id)))
train_mask = sp.sparse.coo_matrix((df.train_int, (df.user_id, df.business_id)))
sparse_train = sparse.multiply(train_mask)
sparse_train.eliminate_zeros()

In [21]:
sparse_train

<1987929x150346 sparse matrix of type '<class 'numpy.float64'>'
	with 5111841 stored elements in Compressed Sparse Row format>

In [3]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

# Load the MovieLens 100k dataset. Only five
# star ratings are treated as positive.
# data = fetch_movielens(min_rating=5.0)

# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(sparse_train, epochs=30, num_threads=2)
# model.fit(['train'], epochs=30, num_threads=4)

# # Evaluate the trained model
# test_precision = precision_at_k(model, sparse_test, k=5).mean()